In [3]:
!pip install transformers datasets
!pip install accelerate -u
!pip install transformers[torch]

import pandas as pd
from datasets import Dataset

# Load MLQA dataset
mlqa_df = pd.read_csv('/content/mlqa_hindi.csv')  # Replace with your actual path
mlqa_dataset = Dataset.from_pandas(mlqa_df)

# Load XQuAD dataset
xquad_df = pd.read_csv('/content/xquad.csv')  # Replace with your actual path
xquad_dataset = Dataset.from_pandas(xquad_df)

# Verify the structure of the datasets
print(mlqa_dataset)
print(xquad_dataset)



Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

no such option: -u
Dataset({
    features: ['context', 'question', 'answer_text', 'answer_start', 'language'],
    num_rows: 5425
})
Dataset({
    features: ['context', 'question', 'answer_text', 'answer_start', 'language'],
    num_rows: 1190
})


In [6]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments, DataCollatorForSeq2Seq

# Load tokenizer and model
model_name = "t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

# Preprocess the datasets
def preprocess_function(examples):
    inputs = [f"question: {q} context: {c}" for q, c in zip(examples["question"], examples["context"])]
    targets = [a for a in examples["answer_text"]]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=128, truncation=True, padding="max_length")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

mlqa_encoded = mlqa_dataset.map(preprocess_function, batched=True)
xquad_encoded = xquad_dataset.map(preprocess_function, batched=True)

# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    evaluation_strategy="steps",
    eval_steps=100,
    save_steps=500,
    logging_dir="./logs",
    logging_steps=10,
    save_total_limit=2,
)

# Data collator
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=mlqa_encoded,
    eval_dataset=xquad_encoded,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

# Train the model
trainer.train()

# Save the model and tokenizer
model.save_pretrained("./results_xquad")
tokenizer.save_pretrained("./results_xquad")


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Map:   0%|          | 0/5425 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3946: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/1190 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Step,Training Loss,Validation Loss
100,0.124500,0.097567
200,0.072000,0.042551
300,0.040000,0.032826
400,0.030200,0.032538
500,0.038400,0.032578
600,0.040900,0.031618
700,0.028900,0.032403
800,0.039500,0.031005
900,0.027900,0.032250
1000,0.030800,0.030415


('./results_xquad/tokenizer_config.json',
 './results_xquad/special_tokens_map.json',
 './results_xquad/spiece.model',
 './results_xquad/added_tokens.json')

In [21]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Load the fine-tuned model and tokenizer
model = T5ForConditionalGeneration.from_pretrained("./results_xquad")
tokenizer = T5Tokenizer.from_pretrained("./results_xquad")

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

def generate_answer(context, question):
    # Prepare the input text
    input_text = f"question: {question} context: {context}"
    inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True, padding=True).to(device)

    # Generate the answer
    outputs = model.generate(
        inputs["input_ids"],
        max_length=50,
        num_beams=5,
        length_penalty=2.0,
        early_stopping=True
    )
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return answer



# Add more test cases if needed
test_cases = [
    {
        "context": "पैंथर्स की डिफ़ेन्स ने लीग में केवल 308 अंक दिए और छठे स्थान पर रहे जबकि 24 इन्टरसेप्शन और चार प्रो बाउलरों के साथ।",
        "question": "पैंथर्स डिफ़ेंस ने कितने अंक दिए?"
    },
    {
        "context":"इंग्लैंड और जर्मनी में कानून और दर्शन का अध्ययन करते हुए, इकबाल ऑल इंडिया मुस्लिम लीग की लंदन शाखा का सदस्य बन गया। वह 1908 में लाहौर वापस आ गया। कानूनी अभ्यास और दार्शनिक कविता के बीच अपने समय को विभाजित करते हुए, इकबाल मुस्लिम लीग में सक्रिय रहे। ",
        "question": "इकबाल किस वर्ष लाहौर लौटे?"
    },
    {
        "context": "ब्रोंकोस ने खेल के अंतिम तीन मिनट में 11 अंक बनाकर डिवीजनल राउंड में पिट्सबर्ग स्टीलर्स को 23–16 से हराया। फिर उन्होंने एएफसी चैम्पियनशिप गेम में न्यू इंग्लैंड के 2-अंक के रूपांतरण के प्रयास के एक पास को इंटरसेप्ट करके मौजूदा सुपर बाउल XLIX चैंपियन न्यू इंग्लैंड पैट्रिओट्स को 17 सेकंड रहते हुए 20–18 से हराया। सीज़न के दौरान मैनिंग की इंटर्सेप्शन की समस्याओं के बावजूद, वे अपने दो प्लेऑफ़ गेम में कोई भी गेम नहीं हारे।",
        "question": "ब्रोंकोस और स्टीलर्स के बीच खेल का अंतिम स्कोर क्या था?"
    },
]

for case in test_cases:
    context = case["context"]
    question = case["question"]
    answer = generate_answer(context, question)
    print(f"Context: {context}")
    print(f"Question: {question}")
    print(f"Answer: {answer}\n")


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Context: पैंथर्स की डिफ़ेन्स ने लीग में केवल 308 अंक दिए और छठे स्थान पर रहे जबकि 24 इन्टरसेप्शन और चार प्रो बाउलरों के साथ।
Question: पैंथर्स डिफ़ेंस ने कितने अंक दिए?
Answer: 308

Context: इंग्लैंड और जर्मनी में कानून और दर्शन का अध्ययन करते हुए, इकबाल ऑल इंडिया मुस्लिम लीग की लंदन शाखा का सदस्य बन गया। वह 1908 में लाहौर वापस आ गया। कानूनी अभ्यास और दार्शनिक कविता के बीच अपने समय को विभाजित करते हुए, इकबाल मुस्लिम लीग में सक्रिय रहे। 
Question: इकबाल किस वर्ष लाहौर लौटे?
Answer: 1908

Context: ब्रोंकोस ने खेल के अंतिम तीन मिनट में 11 अंक बनाकर डिवीजनल राउंड में पिट्सबर्ग स्टीलर्स को 23–16 से हराया। फिर उन्होंने एएफसी चैम्पियनशिप गेम में न्यू इंग्लैंड के 2-अंक के रूपांतरण के प्रयास के एक पास को इंटरसेप्ट करके मौजूदा सुपर बाउल XLIX चैंपियन न्यू इंग्लैंड पैट्रिओट्स को 17 सेकंड रहते हुए 20–18 से हराया। सीज़न के दौरान मैनिंग की इंटर्सेप्शन की समस्याओं के बावजूद, वे अपने दो प्लेऑफ़ गेम में कोई भी गेम नहीं हारे।
Question: ब्रोंकोस और स्टीलर्स के बीच खेल का अंतिम स्कोर क्या था?
Answer: 23–16

